# Introduction, Definitions, and Intuitions  of Privacy and Performance

## What is Split Learning? 

Split learning refers to the process of cutting a Deep Neural Network (DNN) into two or more sections and deploying the first section on-device (e.g. a client's smart-phone) and the second section on the server. In this setting, intermediate activations from the chosen cutting point layer (called split layer) of the DNN are communicated from the client to the server during training. The rest of the network is processed at the server during forward propagation. In turn, during backpropagation, the gradients from the server’s first layer (layer next to the split layer) are communicated back to the client. The rest of the backpropagation occurs at the client. These rounds of communication are continued to finish all the epochs of distributed training ([NoPeek](https://arxiv.org/pdf/2008.09161.pdf) and [split-nn for health](https://arxiv.org/pdf/1812.00564.pdf)). 

Federated learning and split learning share the same goal, which is to avoid sending raw user data to untrusted third parties. However, the method in which they approach this goal is different: federated learning does so by sending only the gradient updates to the server, whereas split learning achieves this by sending the intermediate activations from the split layer, which is an abstracted representation of the input. This is the most important difference between these two schemes, and we won't discuss more details in this session. If you are interested in more extensive comparisons, you can check out these resources: [FL Survey](https://arxiv.org/abs/1912.04977) and [Efficieny Comparison](https://arxiv.org/pdf/1909.09145.pdf).

## How is Split Learning Enhancing Privacy?

Vanilla split learning enhances privacy by sharing only the intermediate activations of the users' data. The intermediate activations are like "smashed data". They have gone through the first few layers of the DNN, and have been stripped of some information they initially had, so in most cases, they have less information than the actual raw input. However, the information lost through the network itself is not necessarily enough to provide any meaningful privacy. Researchers have shown it is possible to invert the intermediate activations and reconstruct the input images with high accuracy [model inversion attack](https://dl.acm.org/doi/abs/10.1145/3359789.3359824). For this reason, split learning is often accompanied by supplementary mechanisms to help mitigate privacy and degrade excessive information two of these mechanisms are [NoPeek](https://arxiv.org/pdf/2008.09161.pdf) and [Shredder](https://arxiv.org/abs/1905.11814).

To re-iterate, we can say split learning, on its own, does not provide any rigorous worst-case privacy guarantees. The privacy enhancement brought by split neural networks and their variations is quantified using information-theoretic notions, and "information degradation". In the rest of this session, we will focus on what we mean by "information-theoretic privacy", and how it can be measured. 

## What is Information-theoretic Privacy?

Information-theoretic privacy is a context-aware privacy notion. Context-aware notions explicitly model the data statistics, unlike context-free solutions (like differential privacy) that assume worst-case dataset statistics and adversaries. In simple words, methods that rely on this privacy notion, show they are decreasing the information content of the user's sensitive data, and are in a sense, removing the sensitive private information. To better understand these methods, we will first go through some simple definitions in information theory, and then quantify what we mean by information.

### Information Theory Basics

#### Information Content (Self-information)
In information theory, the information content or self-information of an event is a quantity arising from the probability of that particular event occurring from a random variable. It can be thought of as an alternative way of expressing probability.  Information content of event $x$ is calculated in bits as:

${\displaystyle \operatorname {I} (x)=-\log _{2}{\left(P\right(x))}.}$


For example, if we toss a fair coin (with $p=1/2$ for heads or tails) and we see heads, the information content in that event of seeing heads is $-\log_2(2^{-1})=1$ bits. 

#### Entropy


 *Entropy* quantifies how much information there is in a *single* random variable ([source](https://machinelearningmastery.com/information-gain-and-mutual-information/)). In other words, entropy is a measure of "surprise", "uncertainty", and "unpredictability". The more unpredictable the outcome of a process is, the higher is the entropy assigned to it.  As an example, consider a biased coin with probability p of landing on heads and probability $1-p$ of landing on tails. The maximum surprise is for $p = 1/2$ when there is no reason to expect one outcome over another, and in this case, a coin flip has an entropy of one bit. The minimum surprise is when $p = 0$ or $p = 1$, when the event is known and the entropy is zero bits ([Wikipedia](https://en.wikipedia.org/wiki/Entropy_(information_theory))). Other values of p give different entropies between zero and one bits. 

Given a discrete random variable ${\displaystyle X}$ (for continous variables the sum is turned into integration), with possible outcomes ${\displaystyle x_{1},...,x_{n}}$, which occur with probability ${\displaystyle \mathrm {P} (x_{1}),...,\mathrm {P} (x_{n})}$, the entropy of ${\displaystyle X}$ is formally defined as:

${\displaystyle \mathrm {H} (X)=-\sum _{i=1}^{n}{\mathrm {P} (x_{i})\log \mathrm {P} (x_{i})}}$



More intuitively, entropy is the **expected value of the information content** of all possible events (outcomes) of a random variable, and it reflects how many bits of information is needed to describe that random variable. In other words, it is roughly the logarithm of the number of values a random variable can take ([source](https://web.stanford.edu/~montanar/RESEARCH/BOOK/partA.pdf)) (in a simple discrete case). In real life, calculating the entropy can be much more convoluted, especially in high dimensional cases.

##### Example 1 
we saw above that the entropy of a fair coin toss ($p=1/2$) is equal to $1$ bit. What is the entropy of tossing $M$ fair coins? 

##### Solution: 
Since there are $2^M$ possible outcomes, the entropy is $-\sum\frac{1}{2^M}\log_2(\frac{1}{2^M}) = M$ bits.

##### Example 2 ([source](https://web.stanford.edu/~montanar/RESEARCH/BOOK/partA.pdf))
A random variable $X$ can take values $0, 1$ with probabilities $p(0) = q$, $p(1) = 1 − q$. Find its Entropy. 

Its entropy is $H(X) = −q\log_2(q) - (1-q)\log_2(1-q)$.  This entropy deminishes when $q = 0$ or $q = 1$ because the outcome is certain, and it is maximal at $q = 1/2$ when the uncertainty of the outcome is maximal.


#### Mutual Information (MI)


Mutual information is calculated between two random variables and measures the reduction in uncertainty for one variable given a known value of the other variable. Intuitively, one might ask, how much does one random variable tell me about another? For example, suppose $X$ represents the roll of a fair 6-sided die, and $Y$ represents whether the roll is even (0 if even, 1 if odd). Clearly, the value of $Y$ tells us something about the value of $X$ and vice versa. That is, these variables share mutual information. On the other hand, if $X$ represents the roll of one fair die, and $Z$ represents the roll of another fair die, then $X$ and $Z$ share no mutual information. The roll of one die does not contain any information about the outcome of the other die ([7](https://people.cs.umass.edu/~elm/Teaching/Docs/mutInf.pdf)). An important theorem from information theory says that the mutual information between two variables is 0 if and only if the two variables are statistically independent. 

The formal definition of the mutual information of two random variables $X$ and $Y$ (discrete, however for continous variables only the sums need to be turend into integrations), whose joint distribution is defined by $P(X, Y)$ is given by:

${\displaystyle \operatorname {I} (X;Y)=\sum _{y\in {\mathcal {Y}}}\sum _{x\in {\mathcal {X}}}{p_{(X,Y)}(x,y)\log {\left({\frac {p_{(X,Y)}(x,y)}{p_{X}(x)\,p_{Y}(y)}}\right)}}}$	
 

 More specifically, this quantifies the "amount of information" (in bits) obtained about one random variable through observing the other random variable. 



##### Relation to Entropy

Mutual information can be equivalently expressed as:

$I(X;Y) = H(X)-H(X|Y) = H(Y)-H(Y|X) = H(X) + H(Y) - H(X,Y) = H(X,Y) - H(X|Y) - H(Y|X) $



The visualization below might help better understand these formulations:

<img width=400  src="mi-venn.png">

What these equations are saying, essentially, is that the mutual information between $X$ and $Y$ is essentially a part of the information in $X$ itself, the part that it shares with $Y$. So, it is the information in $X$ ($H(X)$), minus the part of the information that is in $X$, but is not in $Y$ ($H(X|Y)$). $H(X|Y)$, can be read as "the number of bits of information needed to explain $X$, if we already have $Y$".  

For our split learning scenario, since the mutual information between the user's raw inputs and the intermediate activations that are being sent ($I(X; Y)$) is less than the information content in the image ($I(X; X) = H(X)$), we can allude that "on-average", there is less information, and hence, more privacy. It is very important to keep in mind that since Entropy and Mutual Information are average measures, they do not offer worst-case guarantees.

#### Distance Correlation
Another measure for the dependence between random variables is Distance Correlation (dCor), which is used in the split learning literature ([NoPeek](https://arxiv.org/pdf/2008.09161.pdf)) as well. dCor, similar to Mutual Information, helps capture correlations that are not necessarily linear. dCor takes values $0\leq dCor\leq 1$, where $0$ shows independence and $1$ shows complete dependence. We will learn more about this metric in the following code exercise. If you want to read more on the relation between dCor and MI, checkout [here](http://statweb.stanford.edu/~tibs/reshef/comment.pdf), [here](https://m-clark.github.io/docs/CorrelationComparison.pdf) and [here](http://www.princeton.edu/~dtakahas/publications/Brief%20Bioinform-2013-de%20Siqueira%20Santos). For short, MI needs to be approximated can not be calculated directly in many cases, due to the intractable integrations, however, dCor is easier to compute. 

# Coding Exercise

#### Exercise 1: Event Information Content Vs. Random Variable Entropy 

In the body of the lesson, we saw that the information content of seeing one heads, for a fair coin toss is 1 bit. We also saw that the entropy of the random variable of tossing a fair coin is also 1 bits. Now, consider flipping an unfair coin, with probability $p$ for heads, and $1-p$ for tails. Create a function $information (p)$ that returns the information-content of the event of seeing heads, and entropy for the random variable of tossing the unfair coin. Compare the outputs of $information(\frac{1}{2})$ and $information(\frac{1}{10})$. How do you interpret the results?

##### Solution:

In [1]:
from math import log2
def  information(p):
    inf = -log2(p)
    h = -p*log2(p)-(1-p)*log2(1-p)
    return inf, h

print('p=1/2 : information is: %.3f, entropy is: %.3f bits' % information(1/2))
print('p=1/10: information is: %.3f, entropy is: %.3f bits' % information(1/10))

p=1/2 : information is: 1.000, entropy is: 1.000 bits
p=1/10: information is: 3.322, entropy is: 0.469 bits


In the fair case, the information content of seeing heads is $1.0$ bit, in the unfair case, it is $3.3$ bits. In the latter case, the probability of seeing heads is lower ($0.1$, vs $0.5$), so it is more surprising to see heads, and therefore, it conveys more bits of information ($1.0$ vs $3.2$). However, for the overall random variable, the fair case has $1.0$ bit entropy, but the unfair case has less than half. This makes sense since overall, there is less surprise when we know there is a $90\%$ probability that we see tails, compared to when there is a $50\%$ chance. In other words, the fair coin toss has the highest overall surprise, and therefore the higher entropy. This is also shown in Example 2 above. 